In [1]:
import os, re, signal, socket, select, threading, time, struct
from time import localtime, strftime

HOST, PORT = '127.0.0.1', 8765

try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.connect(('8.8.8.8', 1)) # doesn't even have to be reachable
    HOST = sock.getsockname()[0]
    print(f'HOST IP: {HOST}')
except Exception as error: print(error)
finally: sock.close()

def lines(a): return a.split('\n')

HOST IP: 192.168.4.21


In [2]:
def unpack_uint64(buffer, ii, byteorder='little'): return _unpack(buffer, ii, 8, byteorder)
def unpack_uint32(buffer, ii, byteorder='little'): return _unpack(buffer, ii, 4, byteorder)
def unpack_uint16(buffer, ii, byteorder='little'): return _unpack(buffer, ii, 2, byteorder)

def _unpack(buffer, ii, nbytes, byteorder):
    value = 0
    if byteorder == 'big':
        for i in range(nbytes): value |= buffer[ii+i] << (nbytes-1-i)*8
    elif byteorder == 'little':
        for i in range(nbytes): value |= buffer[ii+i] << i*8
    else: 
        assert False, f'unsupported byteorder: {byteorder}'
        
    return value

In [3]:
def printrsp(tupdata):
    maxint16 = (2**16 / 2)
    for ii in range(0, len(tupdata), 2): 
        cid, value = tupdata[ii], tupdata[ii+1]
        if cid not in i2cStdCmdMap: print(f"Invalid cid: {cid} ({ii} : {tupdata})")
        elif i2cStdCmdMap[cid] == 'Temperature': print(f'Temperature: {((value/10) - 273.15) * (9/5) + 32:.2f} °F')
        elif i2cStdCmdMap[cid] == 'InternalTemperature': print(f'InternalTemperature: {((value/10) - 273.15) * (9/5) + 32:.2f} °F')
        elif i2cStdCmdMap[cid] == 'Voltage': print(f'Voltage: {value * 1e-3:.2f} V')
        elif i2cStdCmdMap[cid] == 'BatteryStatus': print(f'BatteryStatus: {value}')
        elif i2cStdCmdMap[cid] == 'Current': print(f'Current: {value if value < maxint16 else value - (2**16 + 1)} mA')
        elif i2cStdCmdMap[cid] == 'RemainingCapacity': print(f'RemainingCapacity: {value} mAh')
        elif i2cStdCmdMap[cid] == 'FullChargeCapacity': print(f'FullChargeCapacity: {value} mAh')
        elif i2cStdCmdMap[cid] == 'AverageCurrent': print(f'AverageCurrent: {value}')
        elif i2cStdCmdMap[cid] == 'TimeToEmpty': print(f'TimeToEmpty: {value}')
        elif i2cStdCmdMap[cid] == 'TimeToFull': print(f'TimeToFull: {value}')
        elif i2cStdCmdMap[cid] == 'StandbyCurrent': print(f'StandbyCurrent: {value}')
        elif i2cStdCmdMap[cid] == 'StandbyTimeToEmpty': print(f'StandbyTimeToEmpty: {value}')
        elif i2cStdCmdMap[cid] == 'MaxLoadCurrent': print(f'MaxLoadCurrent: {value}')
        elif i2cStdCmdMap[cid] == 'MaxLoadTimeToEmpty': print(f'MaxLoadTimeToEmpty: {value}')
        elif i2cStdCmdMap[cid] == 'RawCoulombCount': print(f'RawCoulombCount: {value}')
        elif i2cStdCmdMap[cid] == 'AveragePower': print(f'AveragePower: {value}')
        elif i2cStdCmdMap[cid] == 'CycleCount': print(f'CycleCount: {value}')
        elif i2cStdCmdMap[cid] == 'RelativeStateOfCharge': print(f'RelativeStateOfCharge: {value}')
        elif i2cStdCmdMap[cid] == 'StateOfHealth': print(f'StateOfHealth: {value}')
        elif i2cStdCmdMap[cid] == 'ChargeVoltage': print(f'ChargeVoltage: {value}')
        elif i2cStdCmdMap[cid] == 'ChargeCurrent': print(f'ChargeCurrent: {value}')
        elif i2cStdCmdMap[cid] == 'DesignCapacity': print(f'DesignCapacity: {value} mAh')
        elif i2cStdCmdMap[cid] == 'OperationStatus':
            bitmap = f'{value:016b}'
            print(f'OperationStatus: ({bitmap[:8]} {bitmap[8:]})')
            print(f'  CFGUPDATE mode: {bitmap[5] == "1"}')
            print(f'  BTP threshold crossed: {bitmap[8] == "1"}')
            print(f'  Smoothing on RemainingCapacity(): {bitmap[9] == "1"}')
            print(f'  Gauge init complete: {bitmap[10] == "1"}')
            print(f'  FCC discharge cycle valid: {bitmap[11] == "1"}')
            print(f'  Voltage below EDV2: {bitmap[12] == "1"}')    
            if bitmap[13:15] == "11": print(f'  Security mode: SEALED')
            elif bitmap[13:15] == "10": print(f'  Security mode: UNSEALED')
            elif bitmap[13:15] == "01": print(f'  Security mode: FULL ACCESS')
            else: print(f'  Security mode: Invalid')
            print(f'  CALIBRATION mode: {"Enabled" if bitmap[15] == "1" else "Disabled"}')
        elif i2cStdCmdMap[cid] == 'CFGUPDATETest': print(f'CFGUPDATETest: {value:016b}')
        elif i2cStdCmdMap[cid] == 'WriteRAMDesignCapacity': print(f'WriteRAMDesignCapacity: {value:016b}')
        elif i2cStdCmdMap[cid] == 'Special': print(f'Special: {value:016b}')
        elif i2cStdCmdMap[cid] == 'Control': print(f'Control: {value:016b}')
        elif i2cStdCmdMap[cid] == 'ManufacturerAccessControl':
            datablk = " ".join([f'{bval:02X}' for bval in value])
            print(f'ManufacturerAccessControl: {datablk}')
 
            ramaddr = (value[1] << 8) | value[0]
            dataval = (value[2] << 8) | value[3]
            if ramaddr in ramAddrMap:
                print(f'  RAM addr: 0x{ramaddr:04X} ({ramAddrMap[ramaddr]}), value: {dataval} (0x{dataval:04X})')
                if ramAddrMap[ramaddr] == "Registers : Operation Config A": printOpsConfigA(dataval)
            else: print(f'  Unrecognized RAM addr: 0x{ramaddr:04X}')
        elif i2cStdCmdMap[cid] == 'Control/SubControl':
            datablk = " ".join([f'{bval:02X}' for bval in value])
            print(f'Control/SubControl: {datablk}')

            ctrlcmd = (value[1] << 8) | value[0]
            if ctrlcmd in ctrlSubCmdMap:
                if ctrlSubCmdMap[ctrlcmd] in ["DEVICE_NUMBER", "HW_VERSION"]:
                    dataval = (value[3] << 8) | value[2]
                    print(f'  I2CCTRLCMD: 0x{ctrlcmd:04X} ({ctrlSubCmdMap[ctrlcmd]}), value: {dataval} (0x{dataval:04X})')
                elif ctrlSubCmdMap[ctrlcmd] == "FW_VERSION":
                    DVBT = {2: 'Device Number', 4: 'Version', 6: 'Build number', 8: 'Firmware type'}
                    print(f'  I2CCTRLCMD: 0x{ctrlcmd:04X} ({ctrlSubCmdMap[ctrlcmd]})')
                    for jj in range(2, 9, 2):
                        dataval = (value[jj] << 8) | value[jj+1]
                        print(f'    {DVBT[jj]}: {dataval} (0x{dataval:04X})')

            else: print(f'  Unrecognized I2CCTRLCMD: 0x{ctrlcmd:04X}')
#}

def printOpsConfigA(value):
    bitmap = f'{value:016b}'
    print(f'Operation Config A: ({bitmap[:8]} {bitmap[8:]})')
    print(f'  TEMPS, Temperature() uses external thermistor: {bitmap[0] == "1"}')
    print(f'  BATG_POL, BAT_GD pin polarity, active on: {"high" if bitmap[2] == "1" else "low"}')
    print(f'  BATG_EN, BATT_GD enabled: {bitmap[3] == "1"}')
    print(f'  SLEEP, fuel gauge SLEEP mode allowed: {bitmap[5] == "1"}')
    print(f'  SLPWAKECHG, accumulate estimated charge on wake from sleep: {bitmap[6] == "1"}')
    print(f'  WRTEMP, host writes temperature (thermistor or internal sensor NOT used): {bitmap[7] == "1"}')
    print(f'  BIEnable, detect battery insertion using TS pin: {bitmap[8] == "1"}')
    print(f'  BI_PUP_EN, battery insertion pin pull up enabled: {bitmap[10] == "1"}')
    print(f'  PFC_CFG1/0, see Section 4.2.1, Pin Function Code (PFC) Descriptions: {bitmap[11:13]}')
    print(f'  WAKE_EN, WK_TH1/0, wake function config, see SLUSCB7 System-Side Data Sheet: {bitmap[13:]}')
#}

i2cStdCmdMap = {
    0x00 : "Control",
    0x02 : "AtRate",
    0x04 : "AtRateTimeToEmpty",
    0x06 : "Temperature",
    0x08 : "Voltage",
    0x0A : "BatteryStatus",
    0x0C : "Current",
    0x10 : "RemainingCapacity",
    0x12 : "FullChargeCapacity",
    0x14 : "AverageCurrent",
    0x16 : "TimeToEmpty",
    0x18 : "TimeToFull",
    0x1A : "StandbyCurrent",
    0x1C : "StandbyTimeToEmpty",
    0x1E : "MaxLoadCurrent",
    0x20 : "MaxLoadTimeToEmpty",
    0x22 : "RawCoulombCount",
    0x24 : "AveragePower",
    0x28 : "InternalTemperature",
    0x2A : "CycleCount",
    0x2C : "RelativeStateOfCharge",
    0x2E : "StateOfHealth",
    0x30 : "ChargeVoltage",
    0x32 : "ChargeCurrent",
    0x34 : "BTPDischargeSet",
    0x36 : "BTPChargeSet",
    0x3A : "OperationStatus",
    0x3C : "DesignCapacity",
    0x3E : "ManufacturerAccessControl",
    0x3F : "Control/SubControl"
}

ctrlSubCmdMap = {
    0x0000 : "CONTROL_STATUS",
    0x0001 : "DEVICE_NUMBER",
    0x0002 : "FW_VERSION",
    0x0003 : "HW_VERSION",
    0x0009 : "BOARD_OFFSET",
    0x000A : "CC_OFFSET",
    0x000B : "CC_OFFSET_SAVE",
    0x000C : "OCV_CMD",
    0x000D : "BAT_INSERT",
    0x000E : "BAT_REMOVE",
    0x0013 : "SET_SNOOZE",
    0x0014 : "CLEAR_SNOOZE",
    0x0015 : "SET_PROFILE_1",
    0x0016 : "SET_PROFILE_2",
    0x0017 : "SET_PROFILE_3",
    0x0018 : "SET_PROFILE_4",
    0x0019 : "SET_PROFILE_5",
    0x001A : "SET_PROFILE_6",
    0x002D : "CAL_TOGGLE",
    0x0030 : "SEALED",
    0x0041 : "RESET",
    0x0080 : "EXIT_CAL",
    0x0081 : "ENTER_CAL",
    0x0090 : "ENTER_CFG_UPDATE",
    0x0091 : "EXIT_CFG_UPDATE_REINIT",
    0x0092 : "EXIT_CFG_UPDATE",
    0x0F00 : "RETURN_TO_ROM"
}

# (see BQ27220 manual: section 3.4 Data Memory Summary)
ramAddrMap = {
    0x929F : "CEDV Profile 1 : Design Capacity",
    0x929D : "CEDV Profile 1 : Full Charge Capacity",
    0x9206 : "Registers : Operation Config A"
}


In [26]:
class LipoTestServer:
    I2CSTDCMD = 1
    I2CCTRLCMD = 2
    I2CRAMREAD = 3
    I2CRAMWRITE = 4
    I2CSPECIAL = 5
    
    def __init__(self, espaddr, testname=None, sinterval=120.0):
        self.doexit = False
        self.espaddr = espaddr
        self.testname = testname
        self.sinterval = sinterval
        self.standalone = False
                
        self.udpsock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.recvthread = threading.Thread(target=self.rsphandler)
        self.recvthread.start()

        signal.signal(signal.SIGINT, self.sighandler)
        signal.signal(signal.SIGTERM, self.sighandler)
    
    def sighandler(self, *args):
        self.doexit = True

    def logrsp(self, tupdata, logname):
        with open(f'lipotests/{logname}', 'a') as logfile:
            tstamp = strftime('%Y-%m-%dT%H:%M:%S', localtime()) 
            for ii in range(0, len(tupdata), 2):
                cid, value = tupdata[ii], tupdata[ii+1]
                if cid not in i2cStdCmdMap: print(f"Invalid tid: {cid} ({ii} : {tupdata})")
                else: logfile.write(f'{tstamp} {i2cStdCmdMap[cid]} {value}\n')
    
    def packStdMsg(self, cmdids, cmdtype):
        assert cmdtype == self.I2CSTDCMD
        chksum, offset = cmdtype, 0
        buffer = bytearray(len(cmdids) + 2)
        struct.pack_into("<B", buffer, offset, cmdtype)
        offset += 1
        
        for cid in cmdids:
            assert cid in i2cStdCmdMap, f"Invalid I2CSTDCMD cmdid: {cid}"
            struct.pack_into("<B", buffer, offset, cid)
            chksum += cid
            offset += 1
        #}
        
        chksum = 0xff - (chksum & 0xff)
        struct.pack_into("<B", buffer, offset, chksum)
        return buffer

    def packCtrlMsg(self, cmdids, cmdtype):
        assert len(cmdids) == 1, "Only one I2CCTRLCMD per send allowed"
        assert cmdids[0] in ctrlSubCmdMap, f"Invalid I2CCTRLCMD cmdid: {cmdids[0]}"
        assert cmdtype == self.I2CCTRLCMD

        buffer = bytearray(4)
        struct.pack_into("<BH", buffer, 0, cmdtype, cmdids[0])
        chksum = 0xff - (sum([int(buffer[i]) for i in range(3)]) & 0xff)
        struct.pack_into("<B", buffer, 3, chksum)
        return buffer        

    def packRamMsg(self, ramaddr, cmdtype, dataval):
        blen = struct.calcsize("<BHB" if cmdtype == self.I2CRAMREAD else "<BHHB")
        
        buffer = bytearray(blen)
        if cmdtype == self.I2CRAMREAD: 
            struct.pack_into("<BH", buffer, 0, cmdtype, ramaddr)
        elif cmdtype == self.I2CRAMWRITE: 
            struct.pack_into("<BHH", buffer, 0, cmdtype, ramaddr, dataval)
        else: assert False, f"Invalid packRamMsg cmdtype: {cmdtype}"
        
        chksum = 0xff - (sum([int(buffer[i]) for i in range(blen-1)]) & 0xff)
        struct.pack_into("<B", buffer, blen-1, chksum)
        return buffer
    
    def unpackrsp(self, rspbuffer):
        data, ii = [], 0
        chksum = (0xFF - (sum(rspbuffer[:-1]) & 0xFF))
        if chksum == rspbuffer[-1]:
            while ii < len(rspbuffer)-1:
                data.append(int(rspbuffer[ii]))
                sz = int(rspbuffer[ii+1])
                ii += 2
                
                if sz == 2: data.append((rspbuffer[ii+1] << 8) | rspbuffer[ii])
                else: data.append([rspbuffer[ii+j] for j in range(sz)])
                ii += sz
        #}
        else: print(f"Invalid MIC: {chksum} : {rspbuffer}")
        return data

    def nxtlogname(self):
        lognum = '0'
        prefix = self.testname if self.testname else "LipoTest"
        pattern = f'{prefix}_{strftime("%Y-%m-%d", localtime())}_(\\d+)\\.log'
        for lsnm in os.listdir(path='lipotests'):
            matchobj = re.match(pattern, lsnm)
            if matchobj: lognum = max(lognum, matchobj.group(1))
        
        return f'{prefix}_{strftime("%Y-%m-%d", localtime())}_{int(lognum) + 1}.log'
    
    def rsphandler(self):
        logname = self.nxtlogname()
        while not self.doexit:
            if not select.select([self.udpsock], [], [], 1.0)[0]: continue
            rspbuffer, _addr = self.udpsock.recvfrom(256) # maxbuffer size 256 bytes
            data = self.unpackrsp(rspbuffer)
            if not self.standalone: self.logrsp(data, logname)
            else: 
                printrsp(data)
                self.doexit = True

    def runStdCmd(self, cmdids, cmdtype=I2CSTDCMD, standalone=False):
        assert cmdtype in [self.I2CSTDCMD, self.I2CCTRLCMD]

        self.standalone = True if cmdtype == self.I2CCTRLCMD else standalone 
        udpbuffer = (self.packStdMsg(cmdids, cmdtype) 
            if cmdtype == self.I2CSTDCMD else 
            self.packCtrlMsg(cmdids, cmdtype))
        
        while not self.doexit:
            initial = time.time()
            self.udpsock.sendto(udpbuffer, self.espaddr)
            while not self.doexit and (self.standalone or (time.time() - initial) < self.sinterval):
                time.sleep(1.0)
        #}

        self.recvthread.join() # Blocks

    def runRamCmd(self, ramaddr, cmdtype, dataval=None):
        assert cmdtype in [self.I2CRAMREAD, self.I2CRAMWRITE]
        if cmdtype == self.I2CRAMWRITE: assert dataval is not None
        
        self.standalone = True
        udpbuffer = self.packRamMsg(ramaddr, cmdtype, dataval)
        
        while not self.doexit:
            initial = time.time()
            self.udpsock.sendto(udpbuffer, self.espaddr)
            while not self.doexit and (self.standalone or (time.time() - initial) < self.sinterval):
                time.sleep(1.0)
        #}

        self.recvthread.join() # Blocks
#}

ESP_UDP_IP = "192.168.4.23"
ESP_UDP_PORT = 4696

ltserver = LipoTestServer(
    espaddr=(ESP_UDP_IP, ESP_UDP_PORT), 
    testname=None,
    sinterval=120.0
)

# STD_CMD_SET = list(i2cStdCmdMap)
STD_CMD_SET = [0x06, 0x28, 0x08, 0x0C, 0x10, 0x12, 0x3A, 0x3C]

# ltserver.runRamCmd(ramaddr=0x929D, cmdtype=LipoTestServer.I2CRAMREAD)
# ltserver.runRamCmd(ramaddr=0x9206, cmdtype=LipoTestServer.I2CRAMWRITE, dataval=0x8484)
# ltserver.runStdCmd(cmdids=[0x0003], cmdtype=LipoTestServer.I2CCTRLCMD)
# ltserver.runStdCmd(cmdids=[0x000C], cmdtype=LipoTestServer.I2CCTRLCMD) # OCV command
ltserver.runStdCmd(cmdids=STD_CMD_SET, cmdtype=LipoTestServer.I2CSTDCMD, standalone=True)


Temperature: 74.57 °F
InternalTemperature: 74.57 °F
Voltage: 3.78 V
Current: -6 mA
RemainingCapacity: 2536 mAh
FullChargeCapacity: 3000 mAh
OperationStatus: (00000000 00100100)
  CFGUPDATE mode: False
  BTP threshold crossed: False
  Smoothing on RemainingCapacity(): False
  Gauge init complete: True
  FCC discharge cycle valid: False
  Voltage below EDV2: False
  Security mode: UNSEALED
  CALIBRATION mode: Disabled
DesignCapacity: 3000 mAh


In [28]:
# Temperature: 78.53 °F
# InternalTemperature: 80.51 °F
# Voltage: 3.84 V
# Current: -810 mA
# RemainingCapacity: 818 mAh
# FullChargeCapacity: 980 mAh
# OperationStatus: (00000000 00110100)
#   CFGUPDATE mode: False
#   BTP threshold crossed: False
#   Smoothing on RemainingCapacity(): False
#   Gauge init complete: True
#   FCC discharge cycle valid: True
#   Voltage below EDV2: False
#   Security mode: UNSEALED
#   CALIBRATION mode: Disabled
# DesignCapacity: 3000 mAh

In [ ]:
0x435, 0x03D4, 0x4e5, 0x505, 0x9226

In [16]:
# Reset sequence

LipoTestServer((ESP_UDP_IP, ESP_UDP_PORT)).runStdCmd(cmdids=[0x0041], cmdtype=LipoTestServer.I2CCTRLCMD)                   # RESET command
LipoTestServer((ESP_UDP_IP, ESP_UDP_PORT)).runRamCmd(ramaddr=0x929F, cmdtype=LipoTestServer.I2CRAMWRITE, dataval=0x03D4)   # Design Capacity = 980 mAh
# # LipoTestServer((ESP_UDP_IP, ESP_UDP_PORT)).runRamCmd(ramaddr=0x929D, cmdtype=LipoTestServer.I2CRAMWRITE, dataval=0x505)    # Full Charge Capacity = 1285 mAh (learned from 3st test (w/EM))
# # LipoTestServer((ESP_UDP_IP, ESP_UDP_PORT)).runRamCmd(ramaddr=0x929D, cmdtype=LipoTestServer.I2CRAMWRITE, dataval=0x4e5)    # Full Charge Capacity = 1253 mAh (learned from 2st full test)
# # LipoTestServer((ESP_UDP_IP, ESP_UDP_PORT)).runRamCmd(ramaddr=0x929D, cmdtype=LipoTestServer.I2CRAMWRITE, dataval=0x435)   # Full Charge Capacity = 1077 mAh (learned from 1st full test)
LipoTestServer((ESP_UDP_IP, ESP_UDP_PORT)).runRamCmd(ramaddr=0x929D, cmdtype=LipoTestServer.I2CRAMWRITE, dataval=0x03D4)  # Full Charge Capacity = 980 mAh
LipoTestServer((ESP_UDP_IP, ESP_UDP_PORT)).runRamCmd(ramaddr=0x9206, cmdtype=LipoTestServer.I2CRAMWRITE, dataval=0x8484)   # Operation Config A (set TEMPS bit)
LipoTestServer((ESP_UDP_IP, ESP_UDP_PORT)).runStdCmd(cmdids=[0x000C], cmdtype=LipoTestServer.I2CCTRLCMD)                   # OCV command (effectively reinits)
 

Control/SubControl: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
ManufacturerAccessControl: 9F 92 03 D4 03 84 0E 74 00 64 0E 9F 00 95 03 63 0F BE 01 3C 09 00 00 0B D7 01 0D 39 01 0D AD 01 10 4D 8D 24
  RAM addr: 0x929F (CEDV Profile 1 : Design Capacity), value: 980 (0x03D4)
ManufacturerAccessControl: 9D 92 03 D4 03 D4 03 84 0E 74 00 64 0E 9F 00 95 03 63 0F BE 01 3C 09 00 00 0B D7 01 0D 39 01 0D AD 01 15 24
  RAM addr: 0x929D (CEDV Profile 1 : Full Charge Capacity), value: 980 (0x03D4)
ManufacturerAccessControl: 06 92 84 84 10 00 00 01 09 00 00 96 00 AF 02 20 00 14 05 00 0A 05 00 32 01 C2 14 14 00 08 09 F6 00 01 91 24
  RAM addr: 0x9206 (Registers : Operation Config A), value: 33924 (0x8484)
Operation Config A: (10000100 10000100)
  TEMPS, Temperature() uses external thermistor: True
  BATG_POL, BAT_GD pin polarity, active on: low
  BATG_EN, BATT_GD enabled: False
  SLEEP, fuel gauge SLEEP mode allowed: True
  SLPWAKECHG, accumula

In [ ]:
# Temperature: 73.13 °F
# InternalTemperature: 74.03 °F
# Voltage: 3.46 V
# Current: -100 mA
# RemainingCapacity: 29 mAh
# FullChargeCapacity: 3000 mAh
# OperationStatus: (00000000 10101100)
#   CFGUPDATE mode: False
#   BTP threshold crossed: True
#   Smoothing on RemainingCapacity(): False
#   Gauge init complete: True
#   FCC discharge cycle valid: False
#   Voltage below EDV2: True
#   Security mode: UNSEALED
#   CALIBRATION mode: Disabled
# DesignCapacity: 980 mAh

In [ ]:
printOpsConfigA(0x0484)
# f'{0x8484:016b}'

In [ ]:
# Configuration:
# 0x929F : "CEDV Profile 1 : Design Capacity" : dataval = 0x03d4 (980 mAh)
# 0x9206 : "Registers : Operation Config A" : dataval = 0x8484 (sets TEMPS flag)

In [ ]:
Running Battery Tests: (1073741824)
MACDataSum: A1
MACDataLen: 24
RAM DesignCapacity: 0B B8 03 84 0E 74 00 64 0E 9F 00 95 03 63 0F BE 01 3C 09 00 00 0B D7 01 0D 39 01 0D AD 01 10 4D
DataBlock:    9F 92 0B B8 03 84 0E 74 00 64 0E 9F 00 95 03 63 0F BE 01 3C 09 00 00 0B D7 01 0D 39 01 0D AD 01 10 4D A1 24
                   "0b b8 03 84 0e 74 00 64 0e 9f 00 95 03 63 0f be 01 3c 09 00 00 0b d7 01 0d 39 01 0d ad 01 10 4d"

In [ ]:
vals = [f"0x{v}" for v in "9F 92 0b b8 03 84 0e 74 00 64 0e 9f 00 95 03 63 0f be 01 3c 09 00 00 0b d7 01 0d 39 01 0d ad 01 10 4d".split()]
vals = [f"0x{v}" for v in "9F 92 04 B0 03 84 0e 74 00 64 0e 9f 00 95 03 63 0f be 01 3c 09 00 00 0b d7 01 0d 39 01 0d ad 01 10 4d".split()]
vals = [f"0x{v}" for v in "9F 92 03 d4 03 84 0e 74 00 64 0e 9f 00 95 03 63 0f be 01 3c 09 00 00 0b d7 01 0d 39 01 0d ad 01 10 4d".split()]
vals = [f"0x{v}" for v in "06 92 84 84 10 00 00 01 09 00 00 96 00 AF 02 20 00 14 05 00 0A 05 00 32 01 C2 14 14 00 08 09 F6 00 01".split()]

total = 0
for i in range(len(vals)):
    total += int(vals[i], 16)

chksum = 0xff - (total & 0xff)
print(f'checksum: {chksum} ({hex(chksum)}), total: {total}')

# hex(0xFF - ((0x40 + sum(vals)) & 0xFF))

In [ ]:
printOpsConfigA(0x0484)

In [ ]:
a = """    {0x00, "Control"},
    {0x02, "AtRate"},
    {0x04, "AtRateTimeToEmpty"},
    {0x06, "Temperature"},
    {0x08, "Voltage"},
    {0x0A, "BatteryStatus"},
    {0x0C, "Current"},
    {0x10, "RemainingCapacity"},
    {0x12, "FullChargeCapacity"},
    {0x14, "AverageCurrent"},
    {0x16, "TimeToEmpty"},
    {0x18, "TimeToFull"},
    {0x1A, "StandbyCurrent"},
    {0x1C, "StandbyTimeToEmpty"},
    {0x1E, "MaxLoadCurrent"},
    {0x20, "MaxLoadTimeToEmpty"},
    {0x22, "RawCoulombCount"},
    {0x24, "AveragePower"},
    {0x28, "InternalTemperature"},
    {0x2A, "CycleCount"},
    {0x2C, "RelativeStateOfCharge"},
    {0x2E, "StateOfHealth"},
    {0x30, "ChargeVoltage"},
    {0x32, "ChargeCurrent"},
    {0x34, "BTPDischargeSet"},
    {0x36, "BTPChargeSet"},
    {0x3A, "OperationStatus"},
    {0x3C, "DesignCapacity"},"""

cnt = 1
for ln in lines(a):
    ln = ln.strip()
    ln = ln.rstrip('},')
    print(f"{ln[1:5]} : {ln[7:]},")
